# Topic Modelling

#### Evan Yathon

This notebook is intended to be run with papermill from the project root.

The purpose of this notebook is to process the review title and content to extract key phrases in each review.  Then key phrases will be used in a regression analysis to find out what is most important for a reviewer in recommending or not recommending the airline.

Topic modelling using LDA will be the tool of choice.

Usage:

`papermill src/ipynbs/topic_modelling.ipynb -p load_path data/cleaned_gw_reviews.csv`

In [1]:
#parameters section for Papermill

load_path = "../../data/cleaned_gw_reviews.csv"

In [2]:
# loading packages

# utils
import pandas as pd
import re
import numpy as np

# NLP
import spacy
import gensim 
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim import models
import pyLDAvis.gensim

# display preference for ipynbs
%matplotlib inline

In [3]:
# ensure the dates are parsed correctly with parse_dates argument
reviews = pd.read_csv(load_path, parse_dates = ["date_of_review", "date_flown"])

In [4]:
reviews.head()

,title,review_value,n_user_reviews,reviewer_name,reviewer_country,date_of_review,review_text,aircraft,traveller_type,seat_type,route,date_flown,seat_comfort_rating,cabin_staff_service_rating,food_and_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_for_money_rating,recommendation
0,"""Seat was fine with enough legroom""",7.0,8 reviews,Sander van Kan,Netherlands,2019-07-01,✅ Trip Verified | Dusseldorf to Berlin. Eurowi...,A319,Couple Leisure,Economy Class,Dusseldorf to Berlin,2019-06-01,4.0,3.0,NaN,1.0,3.0,5.0,yes
1,"""crew were smiling and good""",6.0,8 reviews,Sander van Kan,Netherlands,2019-07-01,✅ Trip Verified | Berlin to Dusseldorf. Eurowi...,A319,Couple Leisure,Economy Class,Berlin to Dusseldorf,2019-06-01,3.0,3.0,NaN,1.0,3.0,5.0,yes
2,"""only two agents available""",1.0,6 reviews,Andrew Maynard,United Kingdom,2017-01-04,Check in process at Cologne very poor. Flight ...,NaN,Couple Leisure,Economy Class,CGN to MAN,2017-01-01,2.0,2.0,NaN,NaN,1.0,2.0,no
3,"""good flight and friendly staff""",7.0,1 reviews,T Steen,Netherlands,2016-09-13,✅ Verified Review | Amsterdam to Stuttgart. G...,NaN,Business,Economy Class,AMS to STR,2016-09-01,5.0,5.0,1.0,NaN,5.0,5.0,yes
4,"""never been treated as badly""",1.0,NaN,Karen Kirner,Austria,2016-08-16,✅ Verified Review | I have been a frequent tr...,NaN,Business,Economy Class,DUS to VIE,2016-08-01,1.0,1.0,NaN,NaN,3.0,1.0,no


### Topic Modelling - Preprocessing

Prior to LDA, preprocessing of the text is necessary.  We need to:
- Tokenize each document (review)
- Minimum token length
- Remove stopwords
- Lemmatize
- Consider only specific parts of speech (nouns, verbs etc.)

Use [`spaCy`](https://spacy.io/) for this.

In [5]:
# load the english spacy model
eng_nlp = spacy.load('en')

In [6]:
# create a function to preprocess the reviews and review titles

def preprocess_text(text, min_token_len = 2, relevant_pos = ["NOUN", "VERB", "ADJ"]):
    """
    Given text, min_token_len, and relevant_pos carry out preprocessing of the text 
    and return a preprocessed string. 
    
    Keyword arguments:
    text (str): the text to be preprocessed
    min_token_len (int): min_token_length required
    relevant_pos (list): a list of relevant pos tags
    
    Returns: (str) the preprocessed text
    """
    # use several regex expressions to preprocess strange characters
    # or unwanted text
    
    # remove verified emoji
    text = re.sub(r"✅", "", text)
    
    # remove verified review and trip verified
    text = re.sub(r"Verified Review|Trip Verified", "", text)
    
    # remove anything that is not a word
    text = re.sub(r"[^\w]", " ", text)
    
    # replace multiple spaces with a single space
    text = re.sub(r"\s+", " ", text)
    
    # remove numbers
    text = re.sub(r"[0-9]+", "", text)
    
    # change all text to lowercase
    text = text.lower()
    
    # remove common terms found in topic modelling output
    # eg/ germanwings, flight
    # these words are not useful for this application as they
    # are common terms found in most reviews, and won't be indicative of 
    # things that influenced their recommendation
    terms = ["flight", "germanwing", "fly", "review"]
    text = re.sub("|".join(terms)," ", text)
    
    # utilize spaCy to tokenize and break text into lemmas/POS:
    # - remove stopwords
    # - reduce words to their lemmas
    # - only keep relevant parts of speech
    # - remove short words less than min_token_len
    
    # spacy will utilize
    doc = eng_nlp(text)
    
    processed_text = ""
    
    for token in doc:
        if not token.is_stop and len(token.text) >= min_token_len and token.pos_ in relevant_pos:
            processed_text += " " + token.lemma_
    
    return processed_text
    

Use the preprocessing function to process all review texts and review titles.

A good check will be to see if the review topics somewhat make sense with the title topics

In [7]:
reviews["clean_review_text"] = reviews["review_text"].apply(preprocess_text, min_token_len = 3)
reviews["clean_title"] = reviews["title"].apply(preprocess_text, min_token_len = 3)

In [8]:
reviews[["clean_review_text", "review_text", "clean_title", "title"]].head(10)

,clean_review_text,review_text,clean_title,title
0,dusseldorf berlin eurowing operate slight del...,✅ Trip Verified | Dusseldorf to Berlin. Eurowi...,seat fine legroom,"""Seat was fine with enough legroom"""
1,berlin dusseldorf eurowing operate different ...,✅ Trip Verified | Berlin to Dusseldorf. Eurowi...,crew smile good,"""crew were smiling and good"""
2,check process cologne poor display delay minu...,Check in process at Cologne very poor. Flight ...,agent available,"""only two agents available"""
3,amsterdam stuttgart good friendly staff airpl...,✅ Verified Review | Amsterdam to Stuttgart. G...,good friendly staff,"""good flight and friendly staff"""
4,frequent traveler year fly distance cabin cla...,✅ Verified Review | I have been a frequent tr...,treat,"""never been treated as badly"""
5,fly london stanste cologne bonn delay hour lo...,✅ Verified Review | Flew Germanwings from Lon...,cramp,"""very cramped"""
6,boyfriend book basic fare hamburg rome fiumic...,My boyfriend and I booked the basic fare from ...,staff friendly,"""staff were friendly"""
7,operation transfer take eurowing read eurowin...,Germanwings operations have now been transferr...,refer eurowing,PLEASE REFER TO EUROWINGS
8,operate barcelona berlin tegel fight time qui...,Lufthansa operated by Germanwings from Barcelo...,great value money,"""great value for money"""
9,bangkok london cologne imagine ryanair hour f...,Germanwings from Bangkok to London via Cologne...,pay little money,"""pay very little money"""


It looks mostly good, but I noticed that in the first review it is removing 'enough'.  This could be valuable in context, but I'll keep digging for now.  It's an issue since 'enough' is a stopword and is being removed.

### Topic Modelling: Dictionary and Document-Term Co-occurrence Matrix

In [9]:
# change the cleaned reviews and cleaned titles to 
# an array, required by corpora.Dictionary
review_corpus = [doc.split() for doc in reviews["clean_review_text"].tolist()]
title_corpus = [doc.split() for doc in reviews["clean_title"].tolist()]

In [10]:
# use gensim dictionary to map between words and integer ids
dct_review = corpora.Dictionary(review_corpus)
dct_title = corpora.Dictionary(title_corpus)

In [11]:
# create the document-term co-occurrence matrices
doc_term_mat_review = [dct_review.doc2bow(doc) for doc in review_corpus]
doc_term_mat_title = [dct_title.doc2bow(doc) for doc in title_corpus]

### Topic Modelling:  LDA Topic Model

Number of topics was tuned using the visualizations below by examining the Intertopic distance map.  When there was no overlap and the topics were separated by a fair amount, the number of topics was chosen.

In [12]:
# create the LDA models
# use a lower alpha, meaning that each document is representative of only a few topics
# do this so that each review doesn't contain similar topics and will have variation
# that we can use to figure out why the recommendation was a yes or no

lda_review = models.LdaModel(corpus = doc_term_mat_review, id2word = dct_review, num_topics = 4, alpha = 0.5, passes = 5)
lda_title = models.LdaModel(corpus = doc_term_mat_title, id2word = dct_title, num_topics = 3, alpha = 0.01, passes = 5)

In [13]:
# view the topics and the words that describe them
lda_review.print_topics()

[(0,
  '0.019*"good" + 0.015*"crew" + 0.015*"seat" + 0.014*"time" + 0.012*"delay" + 0.011*"staff" + 0.011*"cabin" + 0.010*"cologne" + 0.010*"passenger" + 0.009*"minute"'),
 (1,
  '0.014*"time" + 0.013*"service" + 0.012*"seat" + 0.011*"cologne" + 0.011*"delay" + 0.010*"check" + 0.009*"return" + 0.009*"offer" + 0.009*"price" + 0.009*"hour"'),
 (2,
  '0.014*"time" + 0.013*"check" + 0.013*"good" + 0.013*"seat" + 0.012*"airline" + 0.012*"luggage" + 0.009*"pay" + 0.008*"passenger" + 0.008*"charge" + 0.008*"return"'),
 (3,
  '0.016*"seat" + 0.010*"check" + 0.010*"board" + 0.009*"ticket" + 0.009*"good" + 0.008*"return" + 0.008*"hour" + 0.008*"hamburg" + 0.007*"take" + 0.007*"minute"')]

In [14]:
# view the topics and the words that describe them
lda_title.print_topics()

[(0,
  '0.149*"customer" + 0.131*"staff" + 0.099*"friendly" + 0.048*"delay" + 0.028*"problem" + 0.028*"heathrow" + 0.028*"terrible" + 0.028*"food" + 0.028*"airline" + 0.028*"info"'),
 (1,
  '0.775*"customer" + 0.009*"eurowing" + 0.009*"vow" + 0.009*"book" + 0.009*"water" + 0.009*"refuse" + 0.009*"legroom" + 0.009*"refer" + 0.009*"fine" + 0.009*"agent"'),
 (2,
  '0.111*"crew" + 0.111*"friendly" + 0.091*"cabin" + 0.066*"money" + 0.066*"professional" + 0.038*"great" + 0.038*"pay" + 0.038*"value" + 0.038*"little" + 0.038*"service"')]

### Topic Modelling: Visualization

In [15]:
# view the reviews 
pyLDAvis.enable_notebook()
vis_reviews = pyLDAvis.gensim.prepare(lda_review, doc_term_mat_review, dct_review, sort_topics=False)
vis_reviews

C:\Users\evany\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.006390  0.073479       1        1  22.138571
1      0.001890  0.014956       2        1  46.179436
2     -0.079508 -0.038487       3        1  18.470915
3      0.071229 -0.049948       4        1  13.211073, topic_info=    Category       Freq       Term      Total  loglift  logprob
9    Default  62.000000       good  62.000000  30.0000  30.0000
2    Default  47.000000       crew  47.000000  29.0000  29.0000
40   Default  27.000000    luggage  27.000000  28.0000  28.0000
14   Default  78.000000       seat  78.000000  27.0000  27.0000
79   Default  33.000000      cabin  33.000000  26.0000  26.0000
76   Default  29.000000     ticket  29.000000  25.0000  25.0000
141  Default  12.000000    hamburg  12.000000  24.0000  24.0000
61   Default  16.000000     charge  16.000000  23.0000  23.0000
150  Default  20.000000       take  20.000000  22.0000  22.0000
29   Default  58.000000      check  58.000000  21.0000  21.0000
22   Default  39.000000    airline  39.000000  20.0000  20.0000
125  Default  13.000000        use  13.000000  19.0000  19.0000
111  Default  12.000000     lounge  12.000000  18.0000  18.0000
466  Default   6.000000       gold   6.000000  17.0000  17.0000
31   Default  32.000000       cost  32.000000  16.0000  16.0000
38   Default  41.000000       hour  41.000000  15.0000  15.0000
462  Default   7.000000   alliance   7.000000  14.0000  14.0000
58   Default  40.000000      board  40.000000  13.0000  13.0000
450  Default  14.000000      cheap  14.000000  12.0000  12.0000
566  Default  17.000000        cgn  17.000000  11.0000  11.0000
10   Default  22.000000    legroom  22.000000  10.0000  10.0000
200  Default  21.000000      short  21.000000   9.0000   9.0000
35   Default  19.000000       give  19.000000   8.0000   8.0000
286  Default   8.000000        row   8.000000   7.0000   7.0000
471  Default   8.000000       star   8.000000   6.0000   6.0000
183  Default   9.000000       card   9.000000   5.0000   5.0000
328  Default  12.000000     travel  12.000000   4.0000   4.0000
129  Default   5.000000      basic   5.000000   3.0000   3.0000
103  Default  14.000000       gate  14.000000   2.0000   2.0000
72   Default  43.000000      staff  43.000000   1.0000   1.0000
..       ...        ...        ...        ...      ...      ...
286   Topic4   3.588416        row   8.438293   1.1690  -5.3892
843   Topic4   1.529828        one   2.988849   1.3544  -6.2417
471   Topic4   3.614277       star   8.619323   1.1550  -5.3820
848   Topic4   1.506858    similar   2.954099   1.3509  -6.2568
111   Topic4   4.722244     lounge  12.739562   1.0317  -5.1146
695   Topic4   2.146255       roll   4.682004   1.2441  -5.9031
125   Topic4   4.711608        use  13.302923   0.9862  -5.1168
183   Topic4   3.541629       card   9.523153   1.0350  -5.4023
150   Topic4   5.825326       take  20.011536   0.7900  -4.9046
76    Topic4   7.318872     ticket  29.328047   0.6360  -4.6764
6     Topic4   2.924317   eurowing   7.703878   1.0555  -5.5938
14    Topic4  12.755694       seat  78.842697   0.2026  -4.1209
58    Topic4   7.499043      board  40.334198   0.3417  -4.6521
35    Topic4   4.749397       give  19.254009   0.6244  -5.1088
29    Topic4   7.988240      check  58.281933   0.0368  -4.5889
38    Topic4   6.244500       hour  41.345531   0.1339  -4.8352
9     Topic4   7.306059       good  62.600231  -0.1240  -4.6782
122   Topic4   6.497111     return  49.883232  -0.0142  -4.7955
59    Topic4   4.692701       book  25.887669   0.3164  -5.1209
214   Topic4   5.203848  lufthansa  36.384674   0.0794  -5.0175
208   Topic4   3.584062       late  15.559393   0.5559  -5.3904
11    Topic4   5.290098     minute  40.526184  -0.0120  -5.0010
360   Topic4   3.378154    problem  14.208209   0.5876  -5.4495
198   Topic4   4.190216        pay  42.717915  -0.2978  -5.2341
123   Topic4   4.258124    service  50.464775  -0

In [16]:
# view the titles
vis_titles = pyLDAvis.gensim.prepare(lda_title, doc_term_mat_title, dct_title, sort_topics=False)
vis_titles

C:\Users\evany\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.038054  0.077339       1        1  14.999403
1      0.232251 -0.028318       2        1  72.130180
2     -0.194197 -0.049022       3        1  12.870420, topic_info=   Category        Freq          Term       Total  loglift  logprob
43  Default  107.000000      customer  107.000000  30.0000  30.0000
8   Default    5.000000      friendly    5.000000  29.0000  29.0000
9   Default    4.000000         staff    4.000000  28.0000  28.0000
3   Default    3.000000          crew    3.000000  27.0000  27.0000
24  Default    2.000000         cabin    2.000000  26.0000  26.0000
19  Default    2.000000  professional    2.000000  25.0000  25.0000
15  Default    2.000000         money    2.000000  24.0000  24.0000
30  Default    1.000000         delay    1.000000  23.0000  23.0000
17  Default    1.000000        little    1.000000  22.0000  22.0000
29  Default    1.000000       service    1.000000  21.0000  21.0000
16  Default    1.000000         value    1.000000  20.0000  20.0000
14  Default    1.000000         great    1.000000  19.0000  19.0000
18  Default    1.000000           pay    1.000000  18.0000  18.0000
11  Default    1.000000         cramp    1.000000  17.0000  17.0000
4   Default    2.000000          good    2.000000  16.0000  16.0000
31  Default    1.000000       problem    1.000000  15.0000  15.0000
41  Default    1.000000      heathrow    1.000000  14.0000  14.0000
37  Default    1.000000      terrible    1.000000  13.0000  13.0000
36  Default    1.000000          food    1.000000  12.0000  12.0000
20  Default    1.000000       airline    1.000000  11.0000  11.0000
23  Default    1.000000          info    1.000000  10.0000  10.0000
42  Default    1.000000        minute    1.000000   9.0000   9.0000
32  Default    1.000000           don    1.000000   8.0000   8.0000
10  Default    1.000000         treat    1.000000   7.0000   7.0000
21  Default    1.000000        choice    1.000000   6.0000   6.0000
26  Default    1.000000       impress    1.000000   5.0000   5.0000
33  Default    1.000000          late    1.000000   4.0000   4.0000
5   Default    1.000000         smile    1.000000   3.0000   3.0000
38  Default    1.000000   explanation    1.000000   2.0000   2.0000
39  Default    1.000000          help    1.000000   1.0000   1.0000
..      ...         ...           ...         ...      ...      ...
29   Topic3    0.895436       service    1.392429   1.6087  -3.2751
8    Topic3    2.618891      friendly    5.655992   1.2803  -2.2019
17   Topic3    0.895436        little    1.965330   1.2641  -3.2751
26   Topic3    0.223860       impress    1.293754   0.2960  -4.6614
10   Topic3    0.223860         treat    1.293754   0.2960  -4.6614
21   Topic3    0.223860        choice    1.293754   0.2960  -4.6614
33   Topic3    0.223860          late    1.293754   0.2960  -4.6614
20   Topic3    0.223860       airline    1.293754   0.2960  -4.6614
42   Topic3    0.223860        minute    1.293754   0.2960  -4.6614
32   Topic3    0.223860           don    1.293754   0.2960  -4.6614
23   Topic3    0.223860          info    1.293754   0.2960  -4.6614
41   Topic3    0.223860      heathrow    1.293754   0.2960  -4.6614
37   Topic3    0.223860      terrible    1.293754   0.2960  -4.6614
36   Topic3    0.223860          food    1.293754   0.2960  -4.6614
31   Topic3    0.223860       problem    1.293754   0.2960  -4.6614
5    Topic3    0.262627         smile    1.624674   0.2279  -4.5017
38   Topic3    0.223860   explanation    1.626658   0.0670  -4.6614
39   Topic3    0.223860          help    1.626868   0.0668  -4.6614
40   Topic3    0.223860         offer    1.627040   0.0667  -4.6614
22   Topic3    0.223860     satisfied    1.638909   0.0595  -4.6614
25   Topic3    0.223860  unresponsive    1.638909   0.0595  -4.6614
1    Topic3    0.223860       legroom    1.638909   0.0595  -4.6614
2    Topic3    

### Topic Modelling: Assigning each Review a Topic

Create a handy function to help assign topics to the reviews dataframe

In [20]:

def get_correct_topics(lda_doc_topics, num_topics):
    """
    get_correct_topics uses an output LDA object to correctly place
    LDA outputs to their correct topic column number for use in a
    pandas dataframe
    
    Args:
        lda_doc_topics (TransformedCorpus): Document topic probabilities obtained through
                                            the .get_document_topics method of an LDA obj
        num_topics (int) : Number of topics in lda_doc_topics
    Returns:
        pandas dataframe of correctly assigned LDA topics
    """
    # placeholder array with same number of rows as the input 
    # corpus and number of topics for columns
    # initialize all zeros so that topics with missing probabilities
    # will be zero
    
    topic_prob_array = np.zeros((pd.DataFrame(lda_doc_topics).shape[0], num_topics))

    for row_num, row in enumerate(lda_doc_topics):

        # assign each topic to the correct
        # topic[0] is the topic number
        # topic[1] is the corresponding probability
        for topic in row:
            topic_prob_array[row_num, topic[0]] = topic[1]

    topic_prob_df = pd.DataFrame(topic_prob_array)
    
    return topic_prob_df

Get both title and reviews dataframes, concatenate with `reviews`

In [21]:
review_prob_df = get_correct_topics(lda_review.get_document_topics(doc_term_mat_review), 4)
title_prob_df = get_correct_topics(lda_title.get_document_topics(doc_term_mat_title), 3)